%md

# Workshop Azure Databricks
## 14. Deploy and Serve Model using Azure Databricks, MLFlow and Azure ML deployment to ACI or AKS.

<br>
<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>
<br>
Documentation : https://docs.microsoft.com/en-us/azure/machine-learning/

<br>
* Create an Azure ML Workspace
* Build an Azure Container Image for model deployment
* Deploy the model to "staging" using Azure Container Instances (ACI)
* Query the deployed model in "staging"
* Deploy the model to production using Azure Kubernetes Service (AKS)
* Query the deployed model in production
* Update the production deployment
* Clean up the deployments

**Required Libraries via PyPI **:
* `mlflow==1.7.0`  
* `azureml-sdk==1.2.0`

### 1. Create or load an Azure ML Workspace
Before models can be deployed to Azure ML, you must create or obtain an Azure ML Workspace. The `azureml.core.Workspace.create()` function will load a workspace of a specified name or create one if it does not already exist. For more information about creating an Azure ML Workspace, see the [Azure ML Workspace management documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-workspace).

In [0]:
import azureml
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

subscription_id = "TOBEREPLACED" #Votre souscription Azure
resource_group = "AMLworkshop-rg" # Le ressource groupe Azure ML service
workspace_name = "AMLworkshop" # Le nom Azure ML service
workspace_region = "westeurope" # région Azure ML service

In [0]:
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,                      
                      exist_ok=True)


In [0]:
ws = Workspace.from_config()
print('Workspace : ' + ws.name, 
      'Region : ' + ws.location, 
      'Ressource Group : ' + ws.resource_group, sep='\n')

--------------------------------------------------------------------------- 
 UserErrorException Traceback (most recent call last)
 <command-3618905421093301> in <module> 
 ----> 1 ws = Workspace . from_config ( ) 
 2 print('Workspace : ' + ws.name, 
 3 'Region : ' + ws . location , 
 4 'Ressource Group : ' + ws.resource_group, sep='\n')

 /databricks/python/lib/python3.7/site-packages/azureml/core/workspace.py in from_config (path, auth, _logger, _file_name) 
 280 'We could not find config.json in: {} or in its parent directories. ' 
 281 'Please provide the full path to the config file or ensure that ' 
 --> 282 'config.json exists in the parent directories.'.format(normalized_path))
 283 
 284 subscription_id, resource_group, workspace_name = project_info.get_workspace_info(

 UserErrorException : UserErrorException:
	Message: We could not find config.json in: /databricks/driver or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.
	InnerException None
	ErrorResponse 
{
 "error": {
 "code": "UserError",
 "message": "We could not find config.json in: /databricks/driver or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories."
 }
}

### 2. Train the Diabetes Model and build a Container Image for the trained model

In [0]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()  

print('diabetes.keys: ', diabetes.keys())
print()
print('diabetes.data: ', diabetes.data)
print()
print('diabetes.target: ', diabetes.target)

diabetes.keys: dict_keys(['data', 'target', 'DESCR', 'feature_names', 'data_filename', 'target_filename'])

diabetes.data: [[ 0.03807591 0.05068012 0.06169621 ... -0.00259226 0.01990842
 -0.01764613]
 [-0.00188202 -0.04464164 -0.05147406 ... -0.03949338 -0.06832974
 -0.09220405]
 [ 0.08529891 0.05068012 0.04445121 ... -0.00259226 0.00286377
 -0.02593034]
 ...
 [ 0.04170844 0.05068012 -0.01590626 ... -0.01107952 -0.04687948
 0.01549073]
 [-0.04547248 -0.04464164 0.03906215 ... 0.02655962 0.04452837
 -0.02593034]
 [-0.04547248 -0.04464164 -0.0730303 ... -0.03949338 -0.00421986
 0.00306441]]

diabetes.target: [151. 75. 141. 206. 135. 97. 138. 63. 110. 310. 101. 69. 179. 185.
 118. 171. 166. 144. 97. 168. 68. 49. 68. 245. 184. 202. 137. 85.
 131. 283. 129. 59. 341. 87. 65. 102. 265. 276. 252. 90. 100. 55.
 61. 92. 259. 53. 190. 142. 75. 142. 155. 225. 59. 104. 182. 128.
 52. 37. 170. 170. 61. 144. 52. 128. 71. 163. 150. 97. 160. 178.
 48. 270. 202. 111. 85. 42. 170. 200. 252. 113. 143. 51. 52. 210.
 65. 141. 55. 134. 42. 111. 98. 164. 48. 96. 90. 162. 150. 279.
 92. 83. 128. 102. 302. 198. 95. 53. 134. 144. 232. 81. 104. 59.
 246. 297. 258. 229. 275. 281. 179. 200. 200. 173. 180. 84. 121. 161.
 99. 109. 115. 268. 274. 158. 107. 83. 103. 272. 85. 280. 336. 281.
 118. 317. 235. 60. 174. 259. 178. 128. 96. 126. 288. 88. 292. 71.
 197. 186. 25. 84. 96. 195. 53. 217. 172. 131. 214. 59. 70. 220.
 268. 152. 47. 74. 295. 101. 151. 127. 237. 225. 81. 151. 107. 64.
 138. 185. 265. 101. 137. 143. 141. 79. 292. 178. 91. 116. 86. 122.
 72. 129. 142. 90. 158. 39. 196. 222. 277. 99. 196. 202. 155. 77.
 191. 70. 73. 49. 65. 263. 248. 296. 214. 185. 78. 93. 252. 150.
 77. 208. 77. 108. 160. 53. 220. 154. 259. 90. 246. 124. 67. 72.
 257. 262. 275. 177. 71. 47. 187. 125. 78. 51. 258. 215. 303. 243.
 91. 150. 310. 153. 346. 63. 89. 50. 39. 103. 308. 116. 145. 74.
 45. 115. 264. 87. 202. 127. 182. 241. 66. 94. 283. 64. 102. 200.
 265. 94. 230. 181. 156. 233. 60. 219. 80. 68. 332. 248. 84. 200.
 55. 85. 89. 31. 129. 83. 275. 65. 198. 236. 253. 124. 44. 172.
 114. 142. 109. 180. 144. 163. 147. 97. 220. 190. 109. 191. 122. 230.
 242. 248. 249. 192. 131. 237. 78. 135. 244. 199. 270. 164. 72. 96.
 306. 91. 214. 95. 216. 263. 178. 113. 200. 139. 139. 88. 148. 88.
 243. 71. 77. 109. 272. 60. 54. 221. 90. 311. 281. 182. 321. 58.
 262. 206. 233. 242. 123. 167. 63. 197. 71. 168. 140. 217. 121. 235.
 245. 40. 52. 104. 132. 88. 69. 219. 72. 201. 110. 51. 277. 63.
 118. 69. 273. 258. 43. 198. 242. 232. 175. 93. 168. 275. 293. 281.
 72. 140. 189. 181. 209. 136. 261. 113. 131. 174. 257. 55. 84. 42.
 146. 212. 233. 91. 111. 152. 120. 67. 310. 94. 183. 66. 173. 72.
 49. 64. 48. 178. 104. 132. 220. 57.]

#### Train the Data Model

We will uses the `diabetes` dataset in scikit-learn and predicts the progression metric (a quantitative measure of disease progression after one year after) based on BMI, blood pressure, etc. We will uses the scikit-learn ElasticNet linear regression model. We will use MLflow to log  metrics, parameters, artifacts and model.

In [0]:
import os
import warnings
import sys
from random import random
import pandas as pd
import numpy as np
from itertools import cycle
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import lasso_path, enet_path
from sklearn import datasets

# Import mlflow
import mlflow
import mlflow.sklearn

# Load Diabetes datasets
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

# Create pandas DataFrame for sklearn ElasticNet linear_model
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'progression']
data = pd.DataFrame(d, columns=cols)

def train_diabetes(data, in_alpha, in_l1_ratio):
  # Evaluate metrics
  def eval_metrics(actual, pred):
      rmse = np.sqrt(mean_squared_error(actual, pred))
      mae = mean_absolute_error(actual, pred)
      r2 = r2_score(actual, pred)
      return rmse, mae, r2

  warnings.filterwarnings("ignore")
  np.random.seed(40)

  # Split the data into training and test sets. (0.75, 0.25) split.
  train, test = train_test_split(data)

  # The predicted column is "progression" which is a quantitative measure of disease progression one year after baseline
  train_x = train.drop(["progression"], axis=1)
  test_x = test.drop(["progression"], axis=1)
  train_y = train[["progression"]]
  test_y = test[["progression"]]

  if float(in_alpha) is None:
    alpha = 0.05
  else:
    alpha = float(in_alpha)
    
  if float(in_l1_ratio) is None:
    l1_ratio = 0.05
  else:
    l1_ratio = float(in_l1_ratio)
  
  # Start an MLflow run; the "with" keyword ensures we'll close the run even if this cell crashes
  with mlflow.start_run() as run:
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    # Print out ElasticNet model metrics
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    # Log mlflow attributes for mlflow UI
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.sklearn.log_model(lr, "model")
    rand_int = int(random()*10000)
    modelpath = "/dbfs/mlflow/test_diabetes/model-%f-%f-%f" % (alpha, l1_ratio, rand_int)
    mlflow.sklearn.save_model(lr, modelpath)
    print()
    run_id = run.info.run_id
    print('Run ID: ', run_id)
    model_uri = "runs:/" + run_id + "/model"
    print('model_uri: ', model_uri)
    
    return run_id, model_uri
    
run_id, model_uri = train_diabetes(data, 0.01, 1)

Elasticnet model (alpha=0.010000, l1_ratio=1.000000):
 RMSE: 63.24666777876804
 MAE: 51.051828604086325
 R2: 0.3951809598912358

Run ID: 6cc58cc3718a4bbb8c94eb93b1619e21
model_uri: runs:/6cc58cc3718a4bbb8c94eb93b1619e21/model

#### Use MLflow to build a Container Image for the trained model

Use the `mlflow.azureml.build_image` function to build an Azure Container Image for the trained MLflow model. This function also registers the MLflow model with a specified Azure ML workspace. The resulting image can be deployed to Azure Container Instances (ACI) or Azure Kubernetes Service (AKS) for real-time serving.

In [0]:
import mlflow.azureml

model_image, azure_model = mlflow.azureml.build_image(model_uri=model_uri, 
                                                      workspace=ws,
                                                      model_name="model",
                                                      image_name="model",
                                                      description="Sklearn ElasticNet image for predicting diabetes progression",
                                                      synchronous=False)
model_image.wait_for_creation(show_output=True)

Registering model model
2021/02/10 14:18:59 INFO mlflow.azureml: Registered an Azure Model with name: `model` and version: `3`
Creating image
2021/02/10 14:19:02 INFO mlflow.azureml: Building an Azure Container Image with name: `model` and version: `3`
Running..............................................................................................
Succeeded
Image creation operation finished for image model:3, operation "Succeeded"

### 3. Create an ACI webservice deployment

The [ACI platform](https://docs.microsoft.com/en-us/azure/container-instances/) is the recommended environment for staging and developmental model deployments. Using the Azure ML SDK, deploy the Container Image for the trained MLflow model to ACI.

In [0]:
from azureml.core.webservice import AciWebservice, Webservice

dev_webservice_name = "diabetes-db-aci"

dev_webservice_deployment_config = AciWebservice.deploy_configuration(description="Diabetes ML model in ACI")
dev_webservice = Webservice.deploy_from_image(name=dev_webservice_name, image=model_image, deployment_config=dev_webservice_deployment_config, workspace=ws)

dev_webservice.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"

In [0]:
from sklearn import datasets
import pandas as pd
import numpy as np
import requests
import json

diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'progression']
data = pd.DataFrame(d, columns=cols)
sample = data.drop(["progression"], axis=1).iloc[[0]]
                                                 
query_input = sample.to_json(orient='split')
query_input = eval(query_input)
query_input.pop('index', None)

# sending an HTTP request
def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

In [0]:
dev_webservice.scoring_uri
dev_prediction = query_endpoint_example(scoring_uri=dev_webservice.scoring_uri, inputs=query_input)

Sending batch prediction request with inputs: {'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], 'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]]}
Received response: [209.994170046977]

### 4. Deploy the model to production using [Azure Kubernetes Service (AKS)](https://azure.microsoft.com/en-us/services/kubernetes-service/)

In [0]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_cluster_name = "aksclusterfordb" 

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating.......................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None

### 5. Deploy to the model's image to the specified AKS cluster

In [0]:
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core import Image

prod_webservice_name = "diabetes-db-aks"

aks_desc="Diabetes ML model in AKS"

aks_tags={"Data": "Diabetes", 
          "Framework" : "Azure Databricks & TensorFlow", 
          "Purpose" : "AKS Test", 
          "Team" : "France", 
          "Admin" : "Serge"}

prod_webservice_deployment_config = AksWebservice.deploy_configuration(description=aks_desc,tags=aks_tags)

image_name ="model" 

image = Image(name=image_name, workspace=ws)

# Deploy from image
prod_webservice = Webservice.deploy_from_image(workspace = ws, 
                                               name = prod_webservice_name,
                                               image = image,
                                               deployment_config = prod_webservice_deployment_config,
                                               deployment_target = aks_target)

In [0]:
# Wait for the deployment to complete
prod_webservice.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.................
Succeeded
AKS service creation operation finished, operation "Succeeded"

We can evaluate the sample data by sending an HTTP request. Query the AKS webservice's scoring endpoint by sending an HTTP POST request that includes the input vector. The production AKS deployment may require an authorization token (service key) for queries. Include this key in the HTTP request header.

In [0]:
# Test the Webservice

import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print()
  print("Received response: {}".format(preds))
  return preds

In [0]:
import pandas as pd
import numpy as np

X = diabetes.data
y = diabetes.target
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'progression']
data = pd.DataFrame(d, columns=cols)
sample = data.drop(["progression"], axis=1).iloc[[0]]
                                                 
query_input = sample.to_json(orient='split')
query_input = eval(query_input)
query_input.pop('index', None)

#View the webservice prediction

prod_scoring_uri = prod_webservice.scoring_uri
prod_service_key = prod_webservice.get_keys()[0] if len(prod_webservice.get_keys()) > 0 else None
 
print("prod_scoring_uri:") 
print(prod_scoring_uri)
print()
print("prod_service_key:")
print(prod_service_key)
print()
print("prod_prediction1:")
prod_prediction1 = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=query_input)

prod_scoring_uri:
http://20.73.150.63:80/api/v1/service/diabetes-db-aks/score

prod_service_key:
vbjGIA2gdP3MGpof4W6DfunUXlG6by7a

prod_prediction1:
Sending batch prediction request with inputs: {'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], 'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]]}

Received response: [209.994170046977]

### 6. Update the production deployment

Train a new model with different hyperparameters and deploy the new model to production.

In [0]:
import mlflow.azureml

# Train a new model with different hyperparameters
run_id_new, model_uri = train_diabetes(data, 0.01, 0.9)

Elasticnet model (alpha=0.010000, l1_ratio=0.900000):
 RMSE: 63.42512570420111
 MAE: 51.35444421248366
 R2: 0.3917630082341641

Run ID: ebf5536bd03a4f938ecda733a609c92a
model_uri: runs:/ebf5536bd03a4f938ecda733a609c92a/model

In [0]:
# Build a container image for the new trained model
model_image_updated, azure_model_updated = mlflow.azureml.build_image(model_uri=model_uri, 
                                                                      workspace=ws,
                                                                      model_name="model-updated",
                                                                      image_name="model-updated",
                                                                      description="Sklearn ElasticNet image for predicting diabetes progression",
                                                                      synchronous=False)
model_image_updated.wait_for_creation(show_output=True)

Registering model model-updated
2021/02/10 14:41:59 INFO mlflow.azureml: Registered an Azure Model with name: `model-updated` and version: `3`
Creating image
2021/02/10 14:42:02 INFO mlflow.azureml: Building an Azure Container Image with name: `model-updated` and version: `3`
Running...................................................................................................
Succeeded
Image creation operation finished for image model-updated:3, operation "Succeeded"

### 7. Deploy the new model's image to the AKS cluster

Using the [`azureml.core.webservice.AksWebservice.update()`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py#update) function, replace the deployment's existing model image with the new model image.

In [0]:
prod_webservice.update(image=model_image_updated)
prod_webservice.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.................
Succeeded
AKS service creation operation finished, operation "Succeeded"

We can now query the updated model and compare the results.

In [0]:
prod_prediction2 = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=query_input)

Sending batch prediction request with inputs: {'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], 'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]]}

Received response: [192.70109793875872]

In [0]:
print("1st model - Run ID: {} - Prediction= {}".format(run_id, prod_prediction1)) 
print()
print("Updated model - Run ID: {} - Prediction= {}".format(run_id_new, prod_prediction2))

1st model - Run ID: 6cc58cc3718a4bbb8c94eb93b1619e21 - Prediction= [209.994170046977]

Updated model - Run ID: ebf5536bd03a4f938ecda733a609c92a - Prediction= [192.70109793875872]

### 8. Clean up the deployments

In [0]:
dev_webservice.delete()
prod_webservice.delete()
aks_target.delete()